In [1]:
!pip install azureml azureml.core azureml.widgets azureml-dataset-runtime transformers azureml.train

     |████████████████████████████████| 2.2MB 12.2MB/s 
     |████████████████████████████████| 14.1MB 281kB/s 
     |████████████████████████████████| 2.0MB 42.2MB/s 
     |████████████████████████████████| 143kB 47.6MB/s 
     |████████████████████████████████| 1.1MB 37.1MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 727kB 49.3MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 92kB 10.5MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 552kB 48.3MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 153kB 46.6MB/s 
     |████████████████████████████████| 3.2MB 29.9MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 40kB 5.1MB/s 
     |████████████████████████████████| 348kB 30.0MB/s 
     |████████████████████████████████| 39.4MB 113kB/s 


In [1]:
# Azure Connection Details
setup = False
azure = "private"
modelName = "bert-base-uncased" # "openai-gpt" # 'bert-base-uncased'

if azure == "edu":
  subscription_id = "b0cfdf73-f3dd-4fd0-891a-c54130a3d181"
  resource_group = "master"
  workspace_name = "master1"
  computeSize = "small"
  if computeSize == "small":
    computeResource = "master-gpu-12"
  elif computeSize == "big":
    computeResource = "master-gpu-32-1" 

elif azure == "private":
  subscription_id = "93986b83-8c58-4738-abfd-f7d1cbcce9f8"
  resource_group = "master-privat"
  workspace_name = "master-privat"
  computeResource = "cluster-nd6"

repoDir = "/Users/michael/workspaces/MasterThesis"

scriptsPath = os.path.join(repoDir,"scripts")
trainingDataPath = os.path.join(repoDir,"data","embeddings")
modelPath = os.path.join(repoDir,"data","models")
condaFile = os.path.join(repoDir,"environment.yaml")

In [3]:
# Establish Connection to Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="b232d827-1e67-4b06-b634-a6a6785fc4bf")

In [3]:
# Establish Connection to Workspace
from azureml.core import Workspace, Dataset
ws = Workspace(subscription_id, resource_group, workspace_name, auth=interactive_auth)
ws

Workspace.create(name='master-privat', subscription_id='93986b83-8c58-4738-abfd-f7d1cbcce9f8', resource_group='master-privat')

In [4]:
ds = ws.get_default_datastore()
ds

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-62ccc7ca-963e-4c24-867d-ab748217e05b",
  "account_name": "masterprivat1227231575",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [32]:
if setup:
  ds.upload(
      src_dir=trainingDataPath,
      target_path="embeddings",
      overwrite=False,
  )

Uploading an estimated of 11 files
Target already exists. Skipping upload for embeddings/.DS_Store
Target already exists. Skipping upload for embeddings/tokens_climate_1.pt
Target already exists. Skipping upload for embeddings/labels_health_1.pt
Target already exists. Skipping upload for embeddings/dates_health_1.pt
Target already exists. Skipping upload for embeddings/labels_climate_1.pt
Target already exists. Skipping upload for embeddings/dates_climate_1.pt
Target already exists. Skipping upload for embeddings/tokens_health_1.pt
Uploading /Users/michael/workspaces/MasterThesis/data/embeddings/keywordAnalysis_climate_1.pt
Uploaded /Users/michael/workspaces/MasterThesis/data/embeddings/keywordAnalysis_climate_1.pt, 1 files out of an estimated total of 11
Uploading /Users/michael/workspaces/MasterThesis/data/embeddings/leftRightLabels_health_1.pt
Uploaded /Users/michael/workspaces/MasterThesis/data/embeddings/leftRightLabels_health_1.pt, 2 files out of an estimated total of 9
Uploading

In [31]:
if setup:
  from transformers import BertForSequenceClassification
  model = BertForSequenceClassification.from_pretrained(
                                      modelName,
                                      num_labels = 9,
                                      output_attentions = False,
                                      output_hidden_states = False)
  localPretrainedModelPath = modelPath
  model.save_pretrained(localPretrainedModelPath)
  ds.upload(
      src_dir=localPretrainedModelPath,
      target_path=modelName,
      overwrite=False,
  )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
from azureml.core import (
    Experiment,
    Environment,
    ScriptRunConfig,
    Dataset,
    Run
)
from azureml.widgets import RunDetails

In [6]:
ds = Dataset.File.from_files((ws.get_default_datastore(),
                              "embeddings/"))
bertPretrained = Dataset.File.from_files((ws.get_default_datastore(),
                              modelName +"/"))

In [7]:
env = Environment.from_conda_specification(
    name="master-thesis-env",
    file_path=condaFile,
)

In [8]:
category = "climate"
labels = "leftRightPosition"
# leftRightPosition
# partyGroupIdeology

script_run_config = ScriptRunConfig(
    source_directory=scriptsPath,
    script="08_cnn_train.py",
    compute_target=computeResource,
    environment=env,
    arguments=[
        "--data-path",
        ds.as_mount(),
        "--pretrained-model",
        bertPretrained.as_mount(),
        "--epochs",
        5,
        "--batch-size",
        24,
        "--seed",
        4806,
        "--category",
        category,
        "--labels",
        labels,
    ]

)

#        "--learning-rate",
#        2e-5,

#        "--epochs",
#        3,
#        "--learning-rate",
#        0.0029,

#        "--learning-rate",  0.003,
#"--big"
#"--learning-rate",  0.003,

In [10]:
runSingle = Experiment(
    workspace=ws, name="Train_"+category+"_"+labels
).submit(script_run_config)

#RunDetails(runSingle).show()

In [19]:
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, uniform, PrimaryMetricGoal, randint

param_sampling = RandomParameterSampling( {
        'learning-rate': uniform(0.00002, 0.00005)
    }
)
#        'seed' : randint(5000)

early_termination_policy = BanditPolicy(slack_factor=0.05, evaluation_interval=1)

hd_config = HyperDriveConfig(run_config=script_run_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="epoch_train_avg_loss",
                             primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=1)

run = Experiment(workspace=ws, name="HyperParameter_"+category+"_"+labels).submit(hd_config)
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [79]:
## Random Forest Model

categories = ["climate", "health"]
labelsSelection = ["leftRightPosition", "partyGroupIdeology"]

#category = "climate"
#labels = "leftRightPosition"
# leftRightPosition
# partyGroupIdeology
for category in categories:
    for labels in labelsSelection:
        script_run_config = ScriptRunConfig(
            source_directory=scriptsPath,
            script="09_baseline_random_forest.py",
            compute_target=computeResource,
            environment=env,
            arguments=[
                "--data-path",
                ds.as_mount(),
                "--category",
                category,
                "--labels",
                labels
            ]

        )

        runSingle = Experiment(
            workspace=ws, name="Forest_"+category+"_"+labels
        ).submit(script_run_config)

In [11]:
# Export graphs
runName = "HyperParameter_"+category+"_"+labels
print("Load "+runName)
trainExperiment = Experiment(ws, name=runName)
trainRuns = trainExperiment.get_runs(tags={"thesis":1})
trainRun: Run = next(trainRuns)
#print()
trainRunChildren = trainRun.get_children()
trainRunChildren: Run = next(trainRunChildren)
print(trainRunChildren)
#for trainRunChild in trainRunChildren:
#    print(trainRunChild)
#    exit()
#print(trainRunChildren)


Load HyperParameter_climate_leftRightPosition
Run(Experiment: HyperParameter_climate_leftRightPosition,
Id: HD_71d73c5f-8d6a-4043-9f13-0080a6ec131a_17,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: HyperParameter_climate_leftRightPosition,
Id: HD_71d73c5f-8d6a-4043-9f13-0080a6ec131a_16,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: HyperParameter_climate_leftRightPosition,
Id: HD_71d73c5f-8d6a-4043-9f13-0080a6ec131a_15,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: HyperParameter_climate_leftRightPosition,
Id: HD_71d73c5f-8d6a-4043-9f13-0080a6ec131a_14,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: HyperParameter_climate_leftRightPosition,
Id: HD_71d73c5f-8d6a-4043-9f13-0080a6ec131a_13,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: HyperParameter_climate_leftRightPosition,
Id: HD_71d73c5f-8d6a-4043-9f13-0080a6ec131a_12,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: HyperParameter_climate_leftRightPosition